In [ ]:
from google.colab import drive
drive.mount('./MyDrive')

In [ ]:
%cd /content/MyDrive/MyDrive/data/train
!ls

In [ ]:
import keras

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Activation, BatchNormalization, concatenate, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.layers import add
from keras.utils import plot_model
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
from PIL import Image
import os, glob, sys, numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from glob import glob
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import os, re, glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D
from keras.models import load_model

In [ ]:
groups_folder_path = "/content/MyDrive/MyDrive/data/train/"
categories = ["Fe", "Mg"]
 
num_classes = len(categories)
  
image_w = 64
image_h = 64
  
X = []
Y = []
  
for idex, categorie in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    image_dir = groups_folder_path + categorie + '/'
  
    for top, dir, f in os.walk(image_dir):
        for filename in f:
            print(image_dir+filename)
            img = cv2.imread(image_dir+filename)
            img = cv2.resize(img, None, fx=image_w/img.shape[1], fy=image_h/img.shape[0])
            X.append(img/256)
            Y.append(label)
 
X = np.array(X)
Y = np.array(Y)
 
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
xy = (X_train, X_test, Y_train, Y_test)
 
np.save("/content/MyDrive/MyDrive/data/train/numpy/data.npy", xy)

In [ ]:
X_train, X_test, Y_train, Y_test = np.load('/content/MyDrive/MyDrive/data/train/numpy/data.npy', allow_pickle=True)
 
model = Sequential()
model = Sequential()
model.add(Convolution2D(16, 3, 3, activation='relu',input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3,  activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
#model.add(Convolution2D(64, 3, 3))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation = 'softmax'))
plot_model(model, show_shapes=True, to_file='residual_module.png')
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
#model.fit(X_train, Y_train, batch_size=32, epochs=100)
 
#model.save('FE')

In [ ]:
print(np.asarray(X_train).shape[0])

In [ ]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

X_train, X_test, Y_train, Y_test = np.load('/MyDrive/MyDrive/data/train/numpy/Fe.npy', allow_pickle=True)
print(np.asarray(X).shape)
print(np.bincount(Y_train))
print(np.bincount(Y_test))

In [ ]:
X_train = np.array(X_train, dtype=np.float32) / 255

X_test = np.array(X_test, dtype=np.float32) / 255

In [ ]:
growth_rate = 32
theta = 0.5

def DenseBlock(x, num):
    layer_list = [x]
    for i in range(num): # layer 수가 늘어날수록 layer_list에 있는 x에 대해 새로운 x가 concatenate 된다
        x1 = BatchNormalization()(x)
        x1 = Activation('relu')(x1)
        x1 = Convolution2D(growth_rate*4, kernel_size=1, padding='same', use_bias=False)(x1)
        x1 = BatchNormalization()(x1)
        x1 = Activation('relu')(x1)
        x1 = Convolution2D(growth_rate, kernel_size=3, padding='same', use_bias=False)(x1)
        layer_list.append(x1)
        x = concatenate(layer_list, axis=-1) # channel에 대해 layer들을 concatenate
    return x


def Transition_layer(x, name):
    num_ch = int(x.get_shape().as_list()[-1] * theta) # theta 비율만큼 channel 수를 줄임
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Convolution2D(num_ch, kernel_size=3, padding='same', use_bias=False)(x)
    x = AveragePooling2D(pool_size=2, strides=2, padding='same', name=name)(x)
    
    return x

In [ ]:
inputs = Input(shape=(64, 64, 3))
x = Convolution2D(64, kernel_size=7, strides=2, padding='same', use_bias=False)(inputs) # (None,112, 112, 64)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x) # (None,56, 56, 64)

x = DenseBlock(x, 6) # (None,56, 56, 256)
x = Transition_layer(x, 'transition_layer_1') # (None,28, 28, 128)
x = DenseBlock(x, 12) # (None,28, 28, 512)
x = Transition_layer(x, 'transition_layer_2') # (None,14, 14, 256)
x = DenseBlock(x, 24) # (None,14, 14, 1024)
x = Transition_layer(x, 'transition_layer_3') # (None,7, 7, 512)
x = DenseBlock(x, 16) # (None,7, 7, 1024)
x = GlobalAveragePooling2D()(x) # (None,1024)
outputs = Dense(2, activation='softmax')(x) # (None, 10)

model = Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True, to_file='residual_module.png')

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
print(np.asarray(X_train).shape[0])

In [ ]:
model.fit(X_train, Y_train, batch_size=32, epochs=100)